In [1]:
import torch
import pandas as pd

def load_arguments(filename: str):
    with open(filename, "r", encoding="utf8") as f:
        lines = f.readlines()
        headers = lines[0].strip().split("\t")
        records = [l.strip().split("\t") for l in lines[1:]]
    return headers, records


def load_arguments_as_df(filename: str):
    return pd.read_csv(filename, encoding="utf8", sep="\t").set_index("Argument ID")


def load_value_categories(filename: str):
    return pd.read_csv(filename, encoding="utf8", sep="\t").set_index("Argument ID")
# Encode labels to vector space

opposite_pairs = {
    "Self-direction: thought": (0, 1),
    "Self-direction: action": (1, 1),
    "Stimulation": (2, 1),
    "Hedonism": (3, 1),
    "Achievement": (4, 1),
    "Power: dominance": (5, 1),
    "Power: resources": (6, 1),
    "Face": (7, 1),
    "Security: personal": (8, 1),
    "Security: societal": (9, 1),
    "Tradition": (0, -1),
    "Conformity: rules": (1, -1),
    "Conformity: interpersonal": (2, -1),
    "Humility": (3, -1),
    "Benevolence: caring": (4, -1),
    "Benevolence: dependability": (5, -1),
    "Universalism: concern": (6, -1),
    "Universalism: nature": (7, -1),
    "Universalism: tolerance": (8, -1),
    "Universalism: objectivity": (9, -1),
}

pair_dict = {0: ['Self-direction: thought', 'Tradition'], 1: ['Self-direction: action', 'Conformity: rules'], 2: ['Stimulation', 'Conformity: interpersonal'], 3: ['Hedonism', 'Humility'], 4: ['Achievement', 'Benevolence: caring'],
             5: ['Power: dominance', 'Benevolence: dependability'], 6: ['Power: resources', 'Universalism: concern'], 7: ['Face', 'Universalism: nature'], 8: ['Security: personal', 'Universalism: tolerance'], 9: ['Security: societal', 'Universalism: objectivity']}


def encode_label(inputRow):
    output_vector = [0] * len(pair_dict)
    for k, v in pair_dict.items():
        output_vector[k] = inputRow[v[0]] + inputRow[v[1]] * -1
    return output_vector

def decode_label(embeddedVal, threshold=0.3):
    out = [0] * 20
    for i, v in pair_dict.items():
        if(embeddedVal[i] >= threshold):
            out[i] = 1
        elif(embeddedVal[i] <= (-threshold)):
            out[i + 10] = 1
    return out



In [2]:

# from helper import load_arguments, \
#     load_arguments_as_df,\
#     load_value_categories, encode_label

argument_file_path = "data/arguments-training.tsv"
headers, argument_list = load_arguments(argument_file_path)
argument_df = load_arguments_as_df(argument_file_path)

labels_file_path = "data/labels-training.tsv"
labels_df = load_value_categories(labels_file_path)

# validation
val_argument_file_path = "data/arguments-validation.tsv"
val_labels_file_path = "data/labels-validation.tsv"
val_headers, val_argument_list  = load_arguments(val_argument_file_path)
val_argument_df = load_arguments_as_df(val_argument_file_path)
val_labels_df = load_value_categories(val_labels_file_path)

In [51]:
labels_df

,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,Security: societal,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
Argument ID,,,,,,,,,,,,,,,,,,,,
A01002,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
A01005,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
A01006,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
A01007,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0
A01008,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E08016,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
E08017,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,1
E08018,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [3]:
val_argument_df.shape

(1896, 3)

In [4]:
argument_df["Stance"].unique()

array(['in favor of', 'against', 'in favour of'], dtype=object)

In [5]:
labels_df.head(5)

,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,Security: societal,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
Argument ID,,,,,,,,,,,,,,,,,,,,
A01002,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
A01005,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
A01006,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
A01007,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0
A01008,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0


In [6]:
# Clean arguments
argument_df["Stance_encoded"] = argument_df.apply(lambda row: 0 if row["Stance"] == "against" else 1, axis=1)

In [7]:
for s in labels_df.columns:
    print(f'"{s}"')

"Self-direction: thought"
"Self-direction: action"
"Stimulation"
"Hedonism"
"Achievement"
"Power: dominance"
"Power: resources"
"Face"
"Security: personal"
"Security: societal"
"Tradition"
"Conformity: rules"
"Conformity: interpersonal"
"Humility"
"Benevolence: caring"
"Benevolence: dependability"
"Universalism: concern"
"Universalism: nature"
"Universalism: tolerance"
"Universalism: objectivity"


In [8]:
torch.cat(tuple(labels_df.apply(lambda row: torch.Tensor(encode_label(row)).reshape(1,-1), axis=1)), dim=0)

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  ...,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
        ...,
        [ 0.,  0.,  0.,  ..., -1.,  0.,  0.],
        [ 0., -1.,  0.,  ...,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  ...,  0.,  0.,  1.]])

In [9]:
print("Max length conclusion: {}, premise: {}".format(argument_df["Conclusion"].map(lambda x: len(x.split(" "))).max(), argument_df["Premise"].map(lambda x: len(x.split(" "))).max()))
# Conclusion length 64, Premise length 256

Max length conclusion: 35, premise: 133


In [10]:
from transformers import BertTokenizer, BertModel
import numpy as np

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

sentence_model = BertModel.from_pretrained("prajjwal1/bert-small")

def encode_text(text:str, max_length:int):
    encoded = tokenizer.encode_plus(
        text=text,
        add_special_tokens=True,
        truncation=True,
        max_length = max_length,
        padding="max_length",
        return_attention_mask = True,
        return_tensors = "pt"
    )
    output = sentence_model(encoded["input_ids"])
    _, pooled_output = output[:2]
#     return torch.cat((encoded["input_ids"], encoded["attention_mask"]), dim=1)
    return pooled_output[0]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

C:\Users\mehmo\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mehmo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
encode_text(argument_df.iloc[0][0], 64).size()

torch.Size([512])

In [12]:
# Build features
conclusions = list()
with torch.no_grad():
    for i, v in argument_df.iterrows():
        conclusions.append(encode_text(v["Conclusion"], 64).reshape(1,-1))
conclusions = torch.cat(conclusions, dim=0)

arguments = list()
with torch.no_grad():
    for i, v in argument_df.iterrows():
        arguments.append(encode_text(v["Premise"], 256).reshape(1,-1))
arguments = torch.cat(arguments, dim=0)

stance = torch.Tensor(tuple(argument_df["Stance_encoded"])).reshape(-1,1)

In [13]:
stance = stance.reshape(-1,1)

In [12]:
stance.size(),conclusions.size(),arguments.size()

torch.Size([5393, 1])

In [14]:
x_train = torch.cat((conclusions, arguments, stance), dim=1)
y_train = torch.cat(tuple(labels_df.apply(lambda row: torch.Tensor(encode_label(row)).reshape(1,-1), axis=1)), dim=0)

In [15]:
x_train.size()

torch.Size([5393, 1025])

In [16]:
torch.min(x_train, dim=1).values.size()

torch.Size([5393])

In [17]:
torch.max(x_train.abs(), dim=1, keepdim=True)[0].size()

torch.Size([5393, 1])

In [18]:
(x_train/torch.max(x_train.abs(), dim=1, keepdim=True)[0]).size()

torch.Size([5393, 1025])

In [19]:
x_train.size()

torch.Size([5393, 1025])

In [20]:
y_train.size()

torch.Size([5393, 10])

https://stackoverflow.com/questions/68011633/train-multi-output-regression-model-in-pytorch

In [124]:
import torch

class HumanValueRegressor(torch.nn.Module):
    def __init__(self):
        super(HumanValueRegressor, self).__init__()
        self.hidden1 = torch.nn.Linear(1025, 2000)
        self.hidden2 = torch.nn.Linear(2000, 1000)
        self.hidden3 = torch.nn.Linear(1000, 800)
        self.hidden4 = torch.nn.Linear(800, 500)
        self.output = torch.nn.Linear(500, 10)
        
        torch.nn.init.xavier_uniform_(self.hidden1.weight)
        torch.nn.init.zeros_(self.hidden1.bias)
        torch.nn.init.xavier_uniform_(self.hidden2.weight)
        torch.nn.init.zeros_(self.hidden2.bias)
        torch.nn.init.xavier_uniform_(self.hidden3.weight)
        torch.nn.init.zeros_(self.hidden3.bias)
        torch.nn.init.xavier_uniform_(self.hidden4.weight)
        torch.nn.init.zeros_(self.hidden4.bias)
        torch.nn.init.xavier_uniform_(self.output.weight)
        torch.nn.init.zeros_(self.output.bias)
        
    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.relu(self.hidden3(x))
        x = torch.relu(self.hidden4(x))
        x = self.output(x)
        return x/(torch.max(x.abs(), dim=1, keepdim=True)[0])
    
model = HumanValueRegressor()

In [125]:
from tqdm.notebook import tqdm
import random

def training_loop(
    in_features, 
    label_features,
    batch_size,
    epochs,
    model
):
    print("Training...")
    criterion = torch.nn.MSELoss()

    # Create batches
    batches = []
    for i in range(0, len(in_features), batch_size):
        batches.append(
        (
            in_features[i:i+batch_size],
            label_features[i:i+batch_size]
        ))
        
    random.shuffle(batches)

    for i in range(epochs):
        losses = []
        for features, labels in tqdm(batches):
            optimizer.zero_grad()
            out_pred = model(features)
            loss = criterion(out_pred, labels)
#             print(loss)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
    return model

In [126]:
# Training
LR = 0.00001
optimizer = torch.optim.Adam(model.parameters(), LR)
epochs = 200
batch_size = 8

trained_model = training_loop(
    x_train, 
    y_train,
    batch_size,
    epochs,
    model
)

Training...


  0%|          | 0/675 [00:00<?, ?it/s]

epoch 0, loss: 0.35598775344866296


  0%|          | 0/675 [00:00<?, ?it/s]

epoch 1, loss: 0.3535223433706495


...

epoch 197, loss: 0.10197897073847276


  0%|          | 0/675 [00:00<?, ?it/s]

epoch 198, loss: 0.1031316973634616


  0%|          | 0/675 [00:00<?, ?it/s]

epoch 199, loss: 0.10226832565058161


In [147]:
with torch.no_grad():
    x = trained_model(x_train)
x

tensor([[-0.6477,  1.0000,  0.1541,  ..., -0.1709,  0.3198, -0.6226],
        [-0.0176,  0.0421,  0.0164,  ..., -0.3842,  1.0000,  0.1207],
        [ 0.0216, -0.4108,  0.0042,  ...,  0.1730,  0.0289,  1.0000],
        ...,
        [-0.0385, -0.7287, -0.4030,  ..., -0.2124,  0.5450, -0.0332],
        [-0.0181, -0.8372, -0.2881,  ..., -0.2870,  0.7244, -0.1623],
        [ 0.1250,  1.0000, -0.2470,  ..., -0.6285,  0.3921,  0.3053]])

In [148]:
x[0]

tensor([-0.6477,  1.0000,  0.1541, -0.8998,  0.4770, -0.5109, -0.5580, -0.1709,
         0.3198, -0.6226])

In [129]:
y_train

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  ...,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
        ...,
        [ 0.,  0.,  0.,  ..., -1.,  0.,  0.],
        [ 0., -1.,  0.,  ...,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  ...,  0.,  0.,  1.]])

In [149]:
(x - y_train).square().mean()

tensor(0.2216)

In [131]:
len(x),len(y_train)

(5393, 5393)

In [150]:
pred_label = []
true_label = []
for i in range(len(x)):
    pred_label.append(decode_label(x[i]))
    true_label.append(decode_label(y_train[i]))

In [58]:
#After 20 epochs 
print("Classification report:\n", (classification_report(true_label, pred_label)))

Classification report:
               precision    recall  f1-score   support

           0       0.63      0.15      0.24       824
           1       0.47      0.02      0.03      1204
           2       0.00      0.00      0.00       242
           3       0.00      0.00      0.00       159
           4       0.50      0.00      0.01      1092
           5       0.00      0.00      0.00       503
           6       0.00      0.00      0.00       427
           7       0.00      0.00      0.00       367
           8       0.53      0.68      0.60      1726
           9       0.72      0.29      0.42      1438
          10       0.67      0.00      0.01       404
          11       0.40      0.07      0.11       986
          12       0.00      0.00      0.00       202
          13       0.00      0.00      0.00       382
          14       0.32      0.10      0.15       912
          15       0.00      0.00      0.00       699
          16       0.35      1.00      0.52      1883
   

C:\Users\mehmo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mehmo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
#After 100 epochs 
print("Classification report:\n", (classification_report(true_label, pred_label)))

Classification report:
               precision    recall  f1-score   support

           0       0.50      0.59      0.54       824
           1       0.63      0.38      0.48      1204
           2       0.47      0.07      0.12       242
           3       0.36      0.06      0.10       159
           4       0.58      0.30      0.40      1092
           5       0.41      0.16      0.23       503
           6       0.43      0.40      0.41       427
           7       0.26      0.17      0.20       367
           8       0.45      0.86      0.59      1726
           9       0.55      0.68      0.61      1438
          10       0.57      0.23      0.33       404
          11       0.55      0.28      0.37       986
          12       0.52      0.21      0.30       202
          13       0.61      0.27      0.38       382
          14       0.42      0.19      0.26       912
          15       0.50      0.06      0.11       699
          16       0.48      0.78      0.59      1883
   

C:\Users\mehmo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [151]:
#After 200 epochs 
print("Classification report:\n", (classification_report(true_label, pred_label)))

Classification report:
               precision    recall  f1-score   support

           0       0.53      0.67      0.59       824
           1       0.47      0.75      0.58      1204
           2       0.24      0.26      0.25       242
           3       0.21      0.40      0.27       159
           4       0.49      0.60      0.54      1092
           5       0.28      0.53      0.36       503
           6       0.29      0.57      0.39       427
           7       0.27      0.46      0.34       367
           8       0.58      0.78      0.66      1726
           9       0.55      0.78      0.64      1438
          10       0.26      0.71      0.38       404
          11       0.51      0.59      0.55       986
          12       0.16      0.73      0.27       202
          13       0.29      0.72      0.41       382
          14       0.40      0.57      0.47       912
          15       0.40      0.50      0.45       699
          16       0.62      0.78      0.69      1883
   

C:\Users\mehmo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Results 

In [78]:
val_argument_df["Stance_encoded"] = val_argument_df.apply(lambda row: 0 if row["Stance"] == "against" else 1, axis=1)

In [79]:
torch.cat(tuple(val_labels_df.apply(lambda row: torch.Tensor(encode_label(row)).reshape(1,-1), axis=1)), dim=0)

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  1.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0., -1.],
        [ 0.,  1.,  0.,  ...,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  ...,  0.,  0.,  0.]])

In [80]:
encode_text(val_argument_df.iloc[0][0], 64).size()

torch.Size([512])

In [81]:
# Build features
val_conclusions = list()
with torch.no_grad():
    for i, v in val_argument_df.iterrows():
        val_conclusions.append(encode_text(v["Conclusion"], 64).reshape(1,-1))
val_conclusions = torch.cat(val_conclusions, dim=0)

val_arguments = list()
with torch.no_grad():
    for i, v in val_argument_df.iterrows():
        val_arguments.append(encode_text(v["Premise"], 256).reshape(1,-1))
val_arguments = torch.cat(val_arguments, dim=0)

val_stance = torch.Tensor(tuple(val_argument_df["Stance_encoded"])).reshape(-1,1)

In [82]:
val_stance = val_stance.reshape(-1,1)

In [83]:
val_x_train = torch.cat((val_arguments, val_arguments, val_stance), dim=1)
val_y_train = torch.cat(tuple(val_labels_df.apply(lambda row: torch.Tensor(encode_label(row)).reshape(1,-1), axis=1)), dim=0)

In [135]:
with torch.no_grad():
    val_x = trained_model(val_x_train)
val_x

tensor([[ 0.2320, -0.0157, -0.1802,  ..., -1.0000, -0.1863, -0.3216],
        [-0.2206,  0.4501, -0.2606,  ..., -0.5966,  0.0501,  0.0491],
        [-0.1667, -0.3600, -0.1189,  ..., -0.4886,  0.7311, -1.0000],
        ...,
        [ 0.5218,  0.2987,  0.4093,  ..., -0.1852,  1.0000, -0.1895],
        [ 0.3685,  0.3891,  0.0097,  ..., -0.1891,  1.0000, -0.7762],
        [-0.4081,  0.4775,  0.3604,  ...,  0.3109,  0.6973, -0.9407]])

In [136]:
(val_x - val_y_train).square().mean()

tensor(0.4865)

In [144]:
val_pred_label = []
val_true_label = []
for i in range(len(val_x)):
    val_pred_label.append(decode_label(val_x[i],0.1))
    val_true_label.append(decode_label(val_y_train[i],0.1))

In [145]:
#After 200 epochs 
print("Classification report:\n", (classification_report(val_true_label, val_pred_label)))

Classification report:
               precision    recall  f1-score   support

           0       0.10      0.18      0.13       229
           1       0.16      0.14      0.15       426
           2       0.07      0.31      0.11       134
           3       0.04      0.13      0.06        92
           4       0.25      0.27      0.26       407
           5       0.09      0.04      0.05       129
           6       0.06      0.11      0.08        82
           7       0.04      0.04      0.04       126
           8       0.37      0.92      0.53       668
           9       0.35      0.17      0.23       404
          10       0.08      0.58      0.14       150
          11       0.18      0.64      0.28       385
          12       0.02      0.21      0.04        56
          13       0.06      0.60      0.11       116
          14       0.27      0.51      0.35       465
          15       0.12      0.92      0.22       233
          16       0.36      0.88      0.51       637
   

C:\Users\mehmo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
